In [3]:
import requests
import os
import numpy as np
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
from selenium import webdriver

import time
from random import randint
import warnings

import unicodedata
import re
from selenium.webdriver.support.ui import WebDriverWait

from IPython.display import clear_output

In [4]:
def getProfile():
    profile = webdriver.FirefoxProfile()
    profile.set_preference("browser.privatebrowsing.autostart", True)
    return profile

def wait_page_to_refresh(driver, total_houses_initial):
    """
    This function keep reloading the Java Script page until it gets a total number of
    advertisements different from the initial number gotten before entering the function.

    The Viva Real search page display a initial number o advertisements which is the correct total,
    and after a few seconds it display the another number of items. It's necessary to wait page
    to refresh in order to scrap: fist page always repeats the same houses.

    """
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')
#    soup = readparse_html()
    time.sleep(3)

    search_h1 = soup.find('h1') 
    total_houses_final = search_h1.strong.text
    total_houses_final = pd.to_numeric(total_houses_final)
#     total_houses_final = get_total_houses()
    print(f"# of houses: initial = {total_houses_initial}  final = {total_houses_final}")
    return total_houses_final != total_houses_initial


In [5]:
def readparse_html():
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def get_total_houses():
    search_h1 = soup.find('h1') 
    total_houses = search_h1.strong.text
    total_houses = pd.to_numeric(total_houses)
    return total_houses

In [7]:
houses_list= []
source_urls= []
titles = []
urls= []
addresses = []
areas = []
n_bedrooms = []
n_en_suites = []
prices = []
n_bathrooms = []
n_parking_spaces = []
condo_charges = []
page_n=[]
house_n=[]

driver = webdriver.Firefox(firefox_profile=getProfile())

#sleep time
slp = 3

#looping the searches 
for search in range(5):   
    
    pagina = '1'
    
    if search == 0:
        source_url = search_0 = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/"\
        "?pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-"\
        "Santana_de_Parnaiba-Barrios-Aldeia_da_Serra"

    if search == 1:
        source_url = "https://www.vivareal.com.br/venda/sp/santana-de-parnaiba/bairros/"\
        "morada-dos-pinheiros-aldeia-da-serra/?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL"\
        "-Santana_de_Parnaiba-Barrios-Morada_dos_Pinheiros_Aldeia_da_Serra,BR-Sao_Paulo-NULL"\
        "-Santana_de_Parnaiba-Barrios-Morada_das_Flores_Aldeia_da_Serra"
    if search == 2:
        source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/residencial-morada-dos-lagos/"\
        "?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Residencial_Morada_dos_Lagos"\
        ",BR-Sao_Paulo-NULL-Barueri-Barrios-Praca_da_Aldeia_da_Serra_I"

    if search == 3:
        source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/morada-dos-passaros/"\
        "?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Morada_dos_Passaros"\
        ",BR-Sao_Paulo-NULL-Barueri-Barrios-Praca_da_Aldeia_da_Serra_I,BR-Sao_Paulo-NULL-Santana_de_"\
        "Parnaiba-Barrios-Quintas_do_Ingai"
    if search == 4:
        source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/"\
        "estrada-doutor-yojiro-takaoka/condominio_residencial/?__vt=plp:b#onde=BR-Sao_Paulo"\
        "-NULL-Barueri-Barrios-Aldeia_da_Serra|Estrada_Doutor_Yojiro_Takaoka"                    

    
    # get total houses initial if its the first page of the search
            
    #access webpage with search url
    driver.get(source_url)
    time.sleep(1)
    
    #read and parse html page
    soup = readparse_html()
    time.sleep(0.5)

    #get initial number of total houses returned from the search 
    total_houses_initial = get_total_houses()

    #loop until Javascript refreshes the page to get the correct houses
    WebDriverWait(driver, 25).until(lambda driver: wait_page_to_refresh(driver, total_houses_initial))    

    #read and parse html page
    soup = readparse_html()
    time.sleep(slp) 

    #calculate the number of pages to scrap
    number_of_pages = np.ceil((total_houses_initial/36)).astype(int)    
        

      
    #loop every page returned by the search
    for page in range(number_of_pages):
       
        clear_output(wait=True)
        
        pagina = str(page+1)
                            
        if search == 0:
            source_url = search_0 = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/"\
            "?pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-"\
            "Santana_de_Parnaiba-Barrios-Aldeia_da_Serra"

        if search == 1:
            source_url = "https://www.vivareal.com.br/venda/sp/santana-de-parnaiba/bairros/"\
            "morada-dos-pinheiros-aldeia-da-serra/?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL"\
            "-Santana_de_Parnaiba-Barrios-Morada_dos_Pinheiros_Aldeia_da_Serra,BR-Sao_Paulo-NULL"\
            "-Santana_de_Parnaiba-Barrios-Morada_das_Flores_Aldeia_da_Serra"
        if search == 2:
            source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/residencial-morada-dos-lagos/"\
            "?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Residencial_Morada_dos_Lagos"\
            ",BR-Sao_Paulo-NULL-Barueri-Barrios-Praca_da_Aldeia_da_Serra_I"

        if search == 3:
            source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/morada-dos-passaros/"\
            "?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Morada_dos_Passaros"\
            ",BR-Sao_Paulo-NULL-Barueri-Barrios-Praca_da_Aldeia_da_Serra_I,BR-Sao_Paulo-NULL-Santana_de_"\
            "Parnaiba-Barrios-Quintas_do_Ingai"
        if search == 4:
            source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/"\
            "estrada-doutor-yojiro-takaoka/condominio_residencial/?__vt=plp:b#onde=BR-Sao_Paulo"\
            "-NULL-Barueri-Barrios-Aldeia_da_Serra|Estrada_Doutor_Yojiro_Takaoka"                
        
        print(f"page: {str(page+1)}")        
        print(source_url)      
           
        #access webpage with search url
        driver.get(source_url)        
        time.sleep(slp)
        
        soup = readparse_html()
        
        #loop until displaying the real total number of advertisements
        WebDriverWait(driver, 25).until(lambda driver: wait_page_to_refresh(driver, total_houses_initial))                  
        time.sleep(slp)
               
        #read and parse html page with the real number of items
        soup = readparse_html()
        time.sleep(slp)

        #get the real number of items
        total_houses_final = get_total_houses()
        
        #select all house containers in the page
        houses_list = soup.find_all('div', attrs = {'class':'property-card__main-content'})

        #get total number of advertisements in the page
        houses_per_page = len(houses_list)     
        house = 1    

        #loop to read information of each house
        for house in range(houses_per_page):
            page_n.append(page)
            house_n.append(house)
            
            title = houses_list[house].h2.a.text
            titles.append(title)

            if houses_list[house].h2.a['href']:           
                url = 'www.vivareal.com.br'+houses_list[house].h2.a['href']
            else: url=''
            urls.append(url)

            if houses_list[house].h2.span:
                address = houses_list[house].h2.span.text
            else: address = ''
            addresses.append(address)         

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-area'):
                area_m2 = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-area').text
            else: 
                area_m2 = '0'
            areas.append(area_m2)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-room js-property-detail-rooms'):
                bedrooms = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-room js-property-detail-rooms').text
            else: 
                bedrooms = '0'
            n_bedrooms.append(bedrooms)        

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-item-extra js-property-detail-suites'):
                en_suites = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-item-extra js-property-detail-suites').text
            else: en_suites = '0'
            n_en_suites.append(en_suites)

            if houses_list[house].section.div:
                price = houses_list[house].section.div.text
            else: price = '0'
            prices.append(price)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'):
                bathrooms = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom').text
            else: bathrooms='0'
            n_bathrooms.append(bathrooms)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-garage js-property-detail-garages'):
                parking_spaces = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-garage js-property-detail-garages').text
            else: parking_spaces='0'
            n_parking_spaces.append(parking_spaces)        

            house += 1

    page += 1


page: 1
https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/estrada-doutor-yojiro-takaoka/condominio_residencial/?__vt=plp:b#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra|Estrada_Doutor_Yojiro_Takaoka
# of houses: initial = 14  final = 37


In [8]:
driver.close()

In [9]:
aldeia_realestate = pd.DataFrame({ 'page_seq': page_n,
                                'house_seq': house_n,
                                'title': titles,
                                'url': urls,
                                'address': addresses,
                                'area_m2': areas,
                                'bedrooms': n_bedrooms,
                                'en_suites': n_en_suites,
                                'price': prices,
                                'bathrooms': n_bathrooms,
                                'parking_spaces': n_parking_spaces                               
                            })
print(aldeia_realestate.info())
aldeia_realestate.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1581 entries, 0 to 1580
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   page_seq        1581 non-null   int64 
 1   house_seq       1581 non-null   int64 
 2   title           1581 non-null   object
 3   url             1581 non-null   object
 4   address         1581 non-null   object
 5   area_m2         1581 non-null   object
 6   bedrooms        1581 non-null   object
 7   en_suites       1581 non-null   object
 8   price           1581 non-null   object
 9   bathrooms       1581 non-null   object
 10  parking_spaces  1581 non-null   object
dtypes: int64(2), object(9)
memory usage: 136.0+ KB
None


,page_seq,house_seq,title,url,address,area_m2,bedrooms,en_suites,price,bathrooms,parking_spaces
1295,2,2,\n Casa de condomínio com 4 Qua...,www.vivareal.com.br/imovel/casa-de-condominio-...,\n Alameda Sabiá - Morada dos P...,\n\n 400\n \n\nÁrea\n ...,\n\n 4\n \n\n ...,\n\n 4\n \n\n ...,\n \n R$ 1.800.000\n ...,\n\n 7\n \n\n ...,\n\n 4\n \n\n ...
172,4,28,\n Casa de condomínio com 4 Qua...,www.vivareal.com.br/imovel/casa-de-condominio-...,\n Alameda Perseus - Aldeia da ...,\n\n 750\n \n\nÁrea\n ...,\n\n 4\n \n\n ...,\n\n 4\n \n\n ...,\n \n R$ 4.200.000\n ...,\n\n 8\n \n\n ...,\n\n 4\n \n\n ...
1206,10,21,\n Casa de condomínio com 4 Qua...,www.vivareal.com.br/imovel/casa-de-condominio-...,"\n Alameda Mangabeira, 288 - Re...",\n\n 400\n \n\nÁrea\n ...,\n\n 4\n \n\n ...,\n\n 4\n \n\n ...,\n \n R$ 2.000.000\n ...,\n\n 6\n \n\n ...,\n\n 2\n \n\n ...
603,1,27,\n Casa de condomínio com 3 Qua...,www.vivareal.com.br/imovel/casa-de-condominio-...,\n Alameda das Dálias - Morada ...,\n\n 225\n \n\nÁrea\n ...,\n\n 3\n \n\n ...,\n\n 1\n \n\n ...,\n \n R$ 1.150.000\n ...,\n\n 4\n \n\n ...,\n\n 4\n \n\n ...
289,8,1,"\n Casa com 3 Quartos à Venda, ...",www.vivareal.com.br/imovel/casa-3-quartos-alde...,\n Alameda das Sucupiras - Alde...,\n\n 280\n \n\nÁrea\n ...,\n\n 3\n \n\n ...,\n\n 1\n \n\n ...,\n \n R$ 1.150.001\n ...,\n\n 1\n \n\n ...,\n\n 4\n \n\n ...


In [10]:
aux = aldeia_realestate.copy()

In [11]:
aux.to_csv('../data/aldeia_realestate.csv', index=None)